In [1]:
import praw
import pandas as pd
import json
from src.data.validators import RedditComment, RedditPost

LIMIT_SUBREDDIT=10
LIMIT_COMMENTS=25

In [2]:
config = json.load(open('../config.json', 'r'))

In [3]:
reddit = praw.Reddit(
    client_id=config['client_id'], 
    client_secret=config['client_secret'], 
    password=config['password'], 
    user_agent='trend-finder', 
    username=config['username'])

In [4]:
print(reddit.user.me())

noisecape


In [5]:
subreddit = reddit.subreddit('nextfuckinglevel')

In [6]:
data = []

In [7]:
for post in subreddit.hot(limit = LIMIT_SUBREDDIT): 
    reddit_post = RedditPost(
        element_type="post",
        subreddit_id=post.subreddit.id,
        subreddit_display_name=post.subreddit.display_name,
        post_id=post.id,
        title=post.title,
        author=post.author_fullname if hasattr(post, "author_fullname") else None,
        created_utc=post.created_utc,
        selftext=post.selftext,
        score=post.score,
        num_comments=post.num_comments,
        url=post.url,
        gilded=post.gilded,
        num_crossposts=post.num_crossposts,
        over_18=post.over_18,
        permalink=post.permalink,
        upvote_ratio=post.upvote_ratio
    )
    data.append(reddit_post.__dict__)

# Check if the post has comments
    if post.num_comments > 0:
        # Scraping comments for each post
        post.comments.replace_more(limit=25)
        for comment in post.comments.list(): # list() returns list of comments visited in BFS order
            reddit_comment = RedditComment(
                element_type = 'comment',
                comment_id = comment.id,
                author_full_name = comment.author_fullname if hasattr(comment, "author_fullname") else None,
                author_premium = comment.author_premium if hasattr(comment, "author_fullname") else None,
                created_utc = comment.created_utc,
                subreddit_id = comment.subreddit_id,
                num_reports=comment.num_reports,
                score = comment.score,
                gilded = comment.gilded,
                body = comment.body,
                edited = comment.edited,
                permalink = comment.permalink,
                depth = comment.depth,
                controversiality=comment.controversiality,
                parent_id = comment.parent_id if hasattr(comment, 'parent_id') else None
            )
            data.append(reddit_comment.__dict__)

# Create pandas DataFrame for posts and comments
subreddit_df = pd.DataFrame(data)

KeyboardInterrupt: 